In [1]:
# standard imports
import re
import importlib
import inspect
import os
from collections import OrderedDict

# third party imports
import numpy as np
from obspy import read, read_inventory
from obspy.core.event import Origin
from obspy.core.stream import Stream
from obspy.core.trace import Trace
from obspy.geodetics import gps2dist_azimuth
from gmprocess.stationstream import StationStream
from gmprocess.stationtrace import StationTrace
import pandas as pd

# gmprocess imports
from gmprocess.config import get_config
from gmprocess.metrics.rotation import rotate
from gmprocess.io.read import read_data

# local imports
# these are not new methods, but are
# external methods that will need to be slightly modifed
from gmprocess.metrics.metrics_controller import MetricsController

# PGM Controller

In [2]:
"""class MetricsController(object):
    def __init__(self, imts, imcs, timeseries, origin=None, damping=None, bandwidth=None, smooth_type=None):
        self.imts = set([imt.lower() for imt in imts])
        self.imcs = set([imc.lower() for imc in imcs])
        if 'radial_transverse' in self.imcs and origin is None:
            raise Exception('Origin is required for radial_transverse imc')
        self.timeseries = timeseries
        self.origin = origin
        self.config = get_config()
        self.damping = damping
        self.smooth_type = smooth_type
        self.bandwidth = bandwidth
        if damping is None:
            self.damping = self.config['metrics']['sa']['damping']
        if smooth_type is None:
            self.smooth_type = self.config['metrics']['fas']['smoothing']
        if bandwidth is None:
            self.bandwidth = self.config['metrics']['fas']['bandwidth']
        self._available_imts, self._available_imcs = gather_pgms()
        self._step_sets = self.get_steps()
        self._times = self._get_horizontal_time()
        self.pgms = self.execute_steps()
    
    @property
    def step_sets(self):
        return self._step_sets
    
    def get_steps(self):
        pgm_steps = {}
        for imt in self.imts:
            period = None
            percentile = None
            integrate = False
            derivative = False
            baseimt = imt
            if imt == 'pgv' and self.timeseries[0].stats.standard.units == 'acc':
                integrate = True
            elif imt != 'pgv' and self.timeseries[0].stats.standard.units == 'vel':
                derivative = True
                
            if imt.startswith('sa'):
                period = self._parse_period(imt)
                imt = 'sa'
            elif imt.startswith('fas'):
                period = self._parse_period(imt)
                imt = 'fas'
                
            for imc in self.imcs:
                baseimc = imc
                if 'rot' in imc:
                    percentile = self._parse_percentile(imc)
                    if imc.startswith('gm'):
                        imc = 'gmrotd'
                    else:
                        imc = 'rotd'
                # Import
                imt_mod = importlib.import_module('imt.' + imt)
                imc_mod = importlib.import_module('imc.' + imc)
                imt_class = self._get_subclass(inspect.getmembers(imt_mod, inspect.isclass), 'IMT')
                imc_class = self._get_subclass(inspect.getmembers(imc_mod, inspect.isclass), 'IMC')
                # Get Steps
                steps = OrderedDict()
                if derivative:
                    steps['Transform1'] = 'derivative'
                elif integrate:
                    steps['Transform1'] = 'integrate'
                else:
                    steps['Transform1'] = 'null_transform'
                imt_steps = imt_class(imt, imc, period).steps
                imc_steps = imc_class(imc, imt, percentile, period).steps
                steps.update(imt_steps)
                steps.update(imc_steps)
                steps['period'] = period
                steps['percentile'] = percentile
                steps['imc'] = imc
                steps['imt'] = imt
                pgm_steps[baseimt + '_' + baseimc] = steps
        return pgm_steps
    
    def execute_steps(self):
        for idx, imt_imc in enumerate(self.step_sets):
            step_set = self.step_sets[imt_imc]
            period = step_set['period']
            percentile = step_set['percentile']
            if period is not None:
                period = float(period)
            if percentile is not None:
                percentile = float(percentile)
            tseries = self.timeseries.copy()
            # Transform 1
            t1_mod = importlib.import_module('transform.' + step_set['Transform1'])
            t1_cls = self._get_subclass(inspect.getmembers(t1_mod, inspect.isclass), 'Transform')
            t1 = t1_cls(tseries, period, self.damping, self._times).result
            # Transform 2
            t2_mod = importlib.import_module('transform.' + step_set['Transform2'])
            t2_cls = self._get_subclass(inspect.getmembers(t2_mod, inspect.isclass), 'Transform')
            t2 = t2_cls(t1, period, self.damping, self._times).result
            # Rotation
            rot_mod = importlib.import_module('rotation.' + step_set['Rotation'])
            rot_cls = self._get_subclass(inspect.getmembers(rot_mod, inspect.isclass), 'Rotation')
            rot = rot_cls(t2, self.origin).result
            # Transform 3
            t3_mod = importlib.import_module('transform.' + step_set['Transform3'])
            t3_cls = self._get_subclass(inspect.getmembers(t3_mod, inspect.isclass), 'Transform')
            t3 = t3_cls(rot, period, self.damping, self._times).result
            # Combination 1
            c1_mod = importlib.import_module('combination.' + step_set['Combination1'])
            c1_cls = self._get_subclass(inspect.getmembers(c1_mod, inspect.isclass), 'Combination')
            c1 = c1_cls(t3).result
            # Reduction
            red_mod = importlib.import_module('reduction.' + step_set['Reduction'])
            red_cls = self._get_subclass(inspect.getmembers(red_mod, inspect.isclass), 'Reduction')
            red = red_cls(c1, percentile, period, self.smooth_type, self.bandwidth).result
            # Combination 2
            c2_mod = importlib.import_module('combination.' + step_set['Combination2'])
            c2_cls = self._get_subclass(inspect.getmembers(c2_mod, inspect.isclass), 'Combination')
            c2 = c2_cls(red).result
            subdf = self._format(c2, step_set)
            if idx == 0:
                df = subdf
            else:
                df = pd.concat([df, subdf])
        return df
            
    def _format(self, result, steps):

        Creates a dataframe row(rows) structured as:
        imt imc result

        dfdict = OrderedDict() 
        dfdict['IMT'] = []
        dfdict['IMC'] = []
        dfdict['result'] = []
        
        imc = steps['imc']
        imt = steps['imt']
        period = steps['period']
        percentile = steps['percentile']
        if period is not None:
            imt_str = imt.upper() + '(' + period + ')'
        else:
            imt_str = imt.upper()
        if percentile is not None:
            imc_str = imc.upper() + '(' + percentile + ')'
        else:
            imc_str = imc.upper()
            
        if len(result) > 1:
            for r in result:
                dfdict['IMT'] += [imt_str]
                dfdict['IMC'] += [r]
                dfdict['result'] += [result[r]]
        else:
            dfdict['IMT'] += [imt_str]
            dfdict['IMC'] += [imc_str]
            dfdict['result'] += [result['']]
        df = pd.DataFrame(data=dfdict)
        return df
            
    
    def _get_horizontal_time(self):
        for trace in self.timeseries:
            if 'Z' not in trace.stats['channel'].upper():
                times = trace.times()
                return times
    def _get_subclass(self, classes, base_class):
        for cls_tupple in classes:
            if cls_tupple[0] != base_class:
                return cls_tupple[1]
        
    def _parse_period(self, imt):
        period = re.findall('\d+', imt)
        if len(period) > 1:
            period = '.'.join(period)
        else:
            period = period[0]
        return period
    
    def _parse_percentile(self, imc):
        percentile = re.findall('\d+', imc)
        if len(percentile) > 1:
            percentile = '.'.join(percentile)
        else:
            percentile = percentile[0]
        return percentile
"""

"class MetricsController(object):\n    def __init__(self, imts, imcs, timeseries, origin=None, damping=None, bandwidth=None, smooth_type=None):\n        self.imts = set([imt.lower() for imt in imts])\n        self.imcs = set([imc.lower() for imc in imcs])\n        if 'radial_transverse' in self.imcs and origin is None:\n            raise Exception('Origin is required for radial_transverse imc')\n        self.timeseries = timeseries\n        self.origin = origin\n        self.config = get_config()\n        self.damping = damping\n        self.smooth_type = smooth_type\n        self.bandwidth = bandwidth\n        if damping is None:\n            self.damping = self.config['metrics']['sa']['damping']\n        if smooth_type is None:\n            self.smooth_type = self.config['metrics']['fas']['smoothing']\n        if bandwidth is None:\n            self.bandwidth = self.config['metrics']['fas']['bandwidth']\n        self._available_imts, self._available_imcs = gather_pgms()\n        self

In [4]:
datadir = '/Users/hschovanec/Repositories/groundmotion-processing/tests/data/fdsnfetch'
origin = Origin(latitude=47.149, longitude=-122.7266667)
st = read(os.path.join(datadir, 'resp_cor', 'UW.ALCT.--.*.MSEED'))

dummy_stats = {'sensor_serial_number': '12345', 
               'source': 'fdsn',
               'process_time': 'now', 
               'comments': 'No comment', 
               'corner_frequency': 1.0, 
               'source_format': 'fdsn', 
               'instrument_period': 2.0, 
               'structure_type': 'basement', 
               'station_name': 'thisstation', 
               'process_level': 'uncorrected physical units', 
               'instrument': 'dummy', 
               'instrument_damping': 1.0}

st[0].stats.standard = {}
st[0].stats.standard['horizontal_orientation'] = 0.
st[0].stats.standard['units'] = 'acc'
st[0].stats.standard.update(dummy_stats)

st[1].stats.standard = {}
st[1].stats.standard['horizontal_orientation'] = 90.
st[1].stats.standard['units'] = 'acc'
st[1].stats.standard.update(dummy_stats)

st[2].stats.standard = {}
st[2].stats.standard['units'] = 'acc'
st[2].stats.standard.update(dummy_stats)
st[2].stats.standard['horizontal_orientation'] = 500.

inv = read_inventory(os.path.join(datadir, 'inventory.xml'))
stalat, stalon = inv[0][0][0].latitude, inv[0][0][0].longitude

for tr in st:
    tr.stats['coordinates'] = {'latitude': stalat}
    tr.stats['coordinates']['longitude'] = stalon

baz = gps2dist_azimuth(stalat, stalon,
                       origin.latitude, origin.longitude)[1]

st1 = st.copy()
st1[0].stats.channel = st1[0].stats.channel[:-1] + 'N'
st1[1].stats.channel = st1[1].stats.channel[:-1] + 'E'
st1.rotate(method='NE->RT', back_azimuth=baz)
pgms = np.abs(st1.max())

st = st.copy()

"""datadir = '/Users/hschovanec/Repositories/groundmotion-processing/tests/data/geonet/us1000778i/20161113_110259_WTMC_20.V1A'
st = read_data(datadir)[0]"""
imts = ['pgv', 'pga', 'sa2', 'sa1.0', 'sa0.3', 'fas2', 'fas1.0', 'fas0.3', 'arias']
imcs = ['rotd50', 'rotd100.0', 'gmrotd50', 'gmrotd100.0', 'radial_transverse', 'gm', 'am', 'channels', 'greater_of_two_horizontals']
m = MetricsController(imts, imcs, st, origin)
m.pgms

['arias', 'fas', 'pga', 'pgv', 'sa']
['am', 'channels', 'gm', 'gmrotd', 'greater_of_two_horizontals', 'radial_transverse', 'rotd']


,IMT,IMC,result
0,SA(1.0),HNR,0.126378
1,SA(1.0),HNT,0.083318
2,SA(1.0),HNZ,0.093235
0,SA(1.0),GMROTD(50),0.100061
0,SA(1.0),GREATER_OF_TWO_HORIZONTALS,0.129902
0,SA(1.0),GMROTD(100.0),0.103086
0,SA(1.0),HN1,0.129902
1,SA(1.0),HN2,0.088829
2,SA(1.0),HNZ,0.093235
0,SA(1.0),ROTD(50),0.107358
